[Reference](https://amitvkulkarni.medium.com/sensitivity-analysis-using-python-a1b3256bab63)

In [2]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 20.5 MB/s eta 0:00:00


In [10]:
from pulp import *
import pandas as pd

In [4]:
# Step 1: Initialize the Class. We will use the LpMaximize class for optimization
model = LpProblem("Product_Profits",LpMaximize)

In [5]:
# Step 2: Defining variables
A = LpVariable('A', lowBound=0)
B = LpVariable('B', lowBound=0)

In [6]:
# Step 3: Defining objective — profit on products A and B
model += 30 * A + 45 * B

In [7]:
# Step 4: Define the constraints, we will add three constraints
# Constraint 1
model += 3 * A + 12 * B <= 150
# Constraint 2
model += 4 * A + 3 * B <= 47
# Constraint 3
model += 5 * A + 2 * B <= 60

In [8]:
# Step 5: Solve to optimize
model.solve()
print("Model Status:{}".format(LpStatus[model.status]))
print("Objective = ", round(value(model.objective),3))

Model Status:Optimal
Objective =  617.308


In [9]:
# Step 6: Let’s check for the optimal value of A and B
for var in model.variables():
    print(var.name,"=", var.varValue)

A = 2.9230769
B = 11.769231


In [11]:
res = [{'Name':name,'Constraint':const,'Price':const.pi,'Slack': const.slack}
                for name, const in model.constraints.items()]
print(pd.DataFrame(res))

  Name     Constraint     Price      Slack
0  _C1  {A: 3, B: 12}  2.307692  -0.000000
1  _C2   {A: 4, B: 3}  5.769231  -0.000000
2  _C3   {A: 5, B: 2} -0.000000  21.846154


# Building a simulation

In [14]:
lst_constraint1 = list(range(140,151,1))
lst_constraint2 = list(range(45,48,1))
lst_constraint3 = list(range(35,39,1))
def sensitivity_table(lst_constraint1, lst_constraint2, lst_constraint3):
 
    """_summary_
    Args:
        L1 (List): Range of values of constraint 1
        L2 (List): Range of values of constraint 2
        L3 (List): Range of values of constraint 3Returns:
        Int: Returns the optimal value i.e profit
    """
    
    try:
        # Initialize Class, Define Vars., and Objective
        model = LpProblem("Product_Profits",LpMaximize)# Define variables
        A = LpVariable('A', lowBound=0)
        B = LpVariable('B', lowBound=0)# Define Objetive Function: Profit on Product A and B
        model += 30 * A + 45 * B# Constraint 1
        model += 3 * A + 12 * B <= lst_constraint1# Constraint 2
        model += 4 * A + 3 * B <= lst_constraint2# Constraint 3
        model += 5 * A + 2 * B <= lst_constraint3# Solve Model
        model.solve()
        print("Model Status:{}".format(LpStatus[model.status]))
        print("Objective = ", round(value(model.objective),3))
        
        for var in model.variables():
            print(var.name,"=", var.varValue)
            print(f'"lst_constraint1" = {lst_constraint1}, "lst_constraint2" = {lst_constraint2}, "lst_constraint3" = {lst_constraint1}')
        res = [{'Name':name,'Constraint':const,'Price':const.pi,'Slack': const.slack} for name, const in model.constraints.items()]
        print(pd.DataFrame(res))
        return round(value(model.objective),2)
        
    except Exception as e:
        print(f'Simulation error: {e}')
        
        
res = [(p3, p2, p1, sensitivity_table(p1, p2, p3)) for p3 in lst_constraint3 for p2 in lst_constraint2 for p1 in lst_constraint1]

df = pd.DataFrame(res, columns= ['Constraint 3','Constraint 2', 'Constraint 1', 'Objective'])
df_pivot = df.pivot(index = 'Constraint 1', columns = ['Constraint 2', 'Constraint 3'], values = 'Objective')
df_pivot

Model Status:Optimal
Objective =  573.611
A = 2.5925926
"lst_constraint1" = 140, "lst_constraint2" = 45, "lst_constraint3" = 140
B = 11.018519
"lst_constraint1" = 140, "lst_constraint2" = 45, "lst_constraint3" = 140
  Name     Constraint     Price     Slack
0  _C1  {A: 3, B: 12}  3.055556 -0.000000
1  _C2   {A: 4, B: 3} -0.000000  1.574074
2  _C3   {A: 5, B: 2}  4.166667 -0.000000
Model Status:Optimal
Objective =  576.667
A = 2.5555556
"lst_constraint1" = 141, "lst_constraint2" = 45, "lst_constraint3" = 141
B = 11.111111
"lst_constraint1" = 141, "lst_constraint2" = 45, "lst_constraint3" = 141
  Name     Constraint     Price     Slack
0  _C1  {A: 3, B: 12}  3.055556 -0.000000
1  _C2   {A: 4, B: 3} -0.000000  1.444444
2  _C3   {A: 5, B: 2}  4.166667 -0.000000
Model Status:Optimal
Objective =  579.722
A = 2.5185185
"lst_constraint1" = 142, "lst_constraint2" = 45, "lst_constraint3" = 142
B = 11.203704
"lst_constraint1" = 142, "lst_constraint2" = 45, "lst_constraint3" = 142
  Name     Const

Constraint 2,45,46,47,45,46,47,45,46,47,45,46,47
Constraint 3,35,35,35,36,36,36,37,37,37,38,38,38
Constraint 1,,,,,,,,,,,,
140,573.61,573.61,573.61,577.78,577.78,577.78,581.94,581.94,581.94,582.69,586.11,586.11
141,576.67,576.67,576.67,580.83,580.83,580.83,585.00,585.00,585.00,585.00,589.17,589.17
142,579.72,579.72,579.72,583.89,583.89,583.89,587.31,588.06,588.06,587.31,592.22,592.22
143,582.78,582.78,582.78,586.94,586.94,586.94,589.62,591.11,591.11,589.62,595.28,595.28
144,585.83,585.83,585.83,590.00,590.00,590.00,591.92,594.17,594.17,591.92,597.69,598.33
145,588.89,588.89,588.89,593.06,593.06,593.06,594.23,597.22,597.22,594.23,600.00,601.39
146,591.94,591.94,591.94,596.11,596.11,596.11,596.54,600.28,600.28,596.54,602.31,604.44
147,595.00,595.00,595.00,598.85,599.17,599.17,598.85,603.33,603.33,598.85,604.62,607.50
